### Form csv-file for submission

Select users that suits for submission most

In [1]:
from classeslib import train_calendar
from classeslib import persistence_files
from classeslib.predictions_users import PredictionUser, SilentWeeksReader, FutureWeekReader

In [2]:
submission_users_count = 53979

In [3]:
#border_weeks_days = range(train_calendar.last_public_day - 20, train_calendar.last_public_day + 1)
border_weeks_days = range(train_calendar.last_private_day - 20, train_calendar.last_private_day + 1)
border_weeks_days_files = [persistence_files.days_split_catalog + "train." + str(day) + ".csv" 
                           for day in border_weeks_days]
assert len(border_weeks_days_files) == 21

In [5]:
border_weeks_reader = SilentWeeksReader()
for day_file in border_weeks_days_files:
    border_weeks_reader.read_input_file(day_file)

In [8]:
border_weeks_users = {user_id: user for (user_id, user) in border_weeks_reader.users.items() 
                    if len(user.silent_weeks_d3_categories) > 0}
assert len(border_weeks_users) >= submission_users_count

In [9]:
submission_candidates = sorted(
    border_weeks_users.items(),
    key = lambda (iser_id, user): user.views,
    reverse = True)[:submission_users_count * 2]

In [13]:
import shelve
from classeslib.statistics import StatisticsDumper

In [14]:
known_submission_candidates = {}
db = shelve.open(persistence_files.public_train_statistics_db_file)
for (user_id, user) in submission_candidates:
    user_interests_key = StatisticsDumper.user_statistics_key_prefix + user_id
    if not db.has_key(user_interests_key):
        continue
    user_interests = db[user_interests_key]
    user.activeness = user_interests.total_views
    user.all_d3_categories = user_interests.d3_categories
    known_submission_candidates[user_id] = user
db.close()

assert len(known_submission_candidates) >= submission_users_count

In [15]:
for user in known_submission_candidates.values():
    user.nonsilent_weeks_categories = set(user.all_d3_categories.keys()) - user.silent_weeks_d3_categories

In [16]:
with_nonsilent_weeks_interests = {user_id : user for (user_id, user) in known_submission_candidates.items()
                        if len(user.nonsilent_weeks_categories) > 0 and len (user.silent_weeks_d3_categories) > 0}
assert len(with_nonsilent_weeks_interests) >= submission_users_count

In [19]:
submission_users = dict(sorted(
    with_nonsilent_weeks_interests.items(),
    key = lambda (iser_id, user): user.activeness,
    reverse = True)[:submission_users_count])
assert len(submission_users) == submission_users_count

Restore clusters info

In [21]:
from classeslib.clusterization import UsersClaster
import pickle

In [22]:
with open(persistence_files.cluster_model_file, "r") as cluster_file:
    clusterization = pickle.load(cluster_file)

In [ ]:
print sorted(set(clusterization.labels_))

In [24]:
import shelve
cluster_categories = {}
clusters_categories_db = shelve.open(persistence_files.clusters_categories_db_file)
for key in clusters_categories_db.keys():
    cluster_categories[key] = clusters_categories_db[key]
clusters_categories_db.close()

In [ ]:
print persistence_files.clusters_categories_db_file
print sorted(cluster_categories.keys())

In [26]:
from classeslib.features import build_categories_features_matrix

In [27]:
def find_user_cluster(user_id, user_interests, cluster_model, clusters_categories):
    user_features = build_categories_features_matrix(
        d1_categories_sorted, d2_categories_sorted, d3_categories_sorted, {user_id: user_interests})[0]
    return cluster_model.predict(user_features)[0]

In [28]:
(d1_level_statistics,
 d2_level_statistics,
 d3_level_statistics) = StatisticsDumper.restore_statistics(persistence_files.public_train_statistics_db_file)

d1_categories_sorted = sorted(d1_level_statistics.categories_statistics.keys(),
                       key = lambda c : d1_level_statistics.categories_statistics[c],
                       reverse=True)
d2_categories_sorted = sorted(d2_level_statistics.categories_statistics.keys(),
                       key = lambda c : d2_level_statistics.categories_statistics[c],
                       reverse=True)
d3_categories_sorted = sorted(d3_level_statistics.categories_statistics.keys(),
                       key = lambda c : d3_level_statistics.categories_statistics[c],
                       reverse=True)
print len(d3_categories_sorted)

924


In [32]:
submission_users_file = "./db/submission_users.db"
submission_users_interests_file = "./db/submission_users_interests_file.db"
import shelve

In [30]:
submission_users_interests = {}
db = shelve.open(persistence_files.public_train_statistics_db_file)
for user_id in submission_users.keys():
    user_interests_key = StatisticsDumper.user_statistics_key_prefix + user_id
    submission_users_interests[user_id] = db[user_interests_key]
db.close()

In [33]:
submission_users_interests_db = shelve.open(submission_users_interests_file)
for (user_id, user) in submission_users_interests.items():
    submission_users_interests_db[user_id] = user
submission_users_interests_db.close()
#submission_users_interests = {}
#submission_users_interests_db = shelve.open(submission_users_interests_file)
#for user_id in submission_users_interests_db.keys():
#    submission_users_interests[user_id] = submission_users_interests_db[user_id]
#submission_users_interests_db.close()

In [34]:
submission_users_db = shelve.open(submission_users_file)
for (user_id, user) in submission_users.items():
    submission_users_db[user_id] = user
submission_users_db.close()
#submission_users = {}
#submission_users_db = shelve.open(submission_users_file)
#for user_id in submission_users_db.keys():
#    submission_users[user_id] = submission_users_db[user_id]
#submission_users_db.close()
#print len(submission_users_interests)
#print len(submission_users)

Form dataset for Vopal Wabbit

In [35]:
vw_submission_dataset_file = "./vw_submission_dataset"
submission_categories_count = 5

In [37]:
from tqdm import tqdm
from classeslib.vw_datasets import VwDatasetWriter
from classeslib.candidates import get_candidate_categories

In [40]:
with open(vw_submission_dataset_file, 'w') as submission_dataset:
    
    dataset_writer = VwDatasetWriter(submission_dataset)
    
    for (user_id, user) in tqdm(submission_users.items()):  
        
        silent_week_categories = ' '.join(user.silent_weeks_d3_categories)
        assert silent_week_categories != ""
            
        cluster_label = find_user_cluster(
            user_id, submission_users_interests[user_id], clusterization, cluster_categories)
        
        cluster = cluster_categories[str(cluster_label)]
            
        candidates = get_candidate_categories(
            cluster, user_id, user, submission_users_interests, d3_categories_sorted)    
        
        # write candidates rows to test dataset
        for candidate in candidates:
            dataset_writer.write(1, user_id, candidate, silent_week_categories)

100%|██████████| 53979/53979 [13:36<00:00, 66.14it/s]


Run VW on the dataset in test mode

In [41]:
!vw -i vw.track1_model -t vw_submission_dataset -p vw_submission_dataset.out --quiet

Select predicted categories

In [43]:
from classeslib.candidates import get_users_predicted_categories 

In [45]:
y_pred = get_users_predicted_categories(
    vw_submission_dataset_file, "./vw_submission_dataset.out", d3_categories_sorted)

Create submission file

In [46]:
submission_predictions_file = "predicted_submission.csv"

In [48]:
submission_header = "user_id,id3_1,id3_2,id3_3,id3_4,id3_5\n"
with open(submission_predictions_file, "w") as submision_file:
    submision_file.write(submission_header)
    for (user_id, categories) in y_pred.items():
        assert len(categories) == 5
        line = user_id +"," + ",".join(categories) + "\n"
        submision_file.write(line)